<a href="https://colab.research.google.com/github/aboelela924/LeagueOfLegendsAnalytics/blob/master/LeagueOfLegendsAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/LOLDataset/matches.jsonl.7z /content

In [ ]:
!apt install p7zip-full

In [ ]:
!7z e matches.jsonl.7z

In [ ]:
# ! pip install -q kaggle

In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets download -d d4sein/league-of-legends-patch-109

In [ ]:
# !unzip /content/league-of-legends-patch-109.zip

In [ ]:
!rm /content/matchesSmall.json
!head /content/matches.jsonl  -n 50000 >> /content/matchesSmall.json

In [ ]:
!head /content/matches.jsonl -n 1

In [ ]:
# !tail /content/matchesSmall.json -n 5000 

In [ ]:
# MYFILE="/content/matchesSmall.json"

# # read the file into a list of lines
# lines = open(MYFILE, 'r').readlines()

# # now edit the last line of the list of lines
# new_last_line = ("}]")
# print(new_last_line)
# lines[-1] = new_last_line

# # now write the modified list back out to the file
# open(MYFILE, 'w').writelines(lines)

In [ ]:
spark = SparkSession.builder.master("local[1]").appName("League of legends analytics").getOrCreate()

In [ ]:
matches = spark.read.text("/content/matches.jsonl")

<h5><b>**Game Counter**</b></h5>

In [ ]:
import timeit

start = timeit.default_timer()
gameCountCalcOp = matches.rdd.map(lambda match: ("_", 1))
gameCountCalcOp = gameCountCalcOp.reduceByKey(lambda a,b: a+b)
gameCountCalcOp = gameCountCalcOp.collect()
gameCountCalcOp = gameCountCalcOp[0][1]
gameCount = spark.sparkContext.broadcast(gameCountCalcOp)
stop = timeit.default_timer()
print('Time: ', stop - start)  

In [ ]:
print(gameCountCalcOp)

<h5><b>**CHAMPION WIN RATE**</b></h5>

In [ ]:
import json
# def reader(x):
#     return json.loads(x.value)
    
def mapChampionIdToWinLoss(x):
    x = json.loads(x.value)
    participants = x["participants"]
    result = []
    for participant in participants:
        championId = participant["championId"]
        isWin = participant["stats"]["win"]
        if isWin:
            result.append((championId, (1, 0)))
        else:
            result.append((championId, (0, 1)))
    return result
def calcRate(x):
    rate = (x[1][0]/ (x[1][0]+x[1][1])) * 100
    rate = f"{rate}%"
    return (x[0], rate)

In [ ]:

start = timeit.default_timer()
if os.path.isdir("/content/winResult"):
    !rm -r /content/winResult
if os.path.isdir("/content/winResultPickled"):
    !rm -r /content/winResultPickled
# winRate = matches.rdd.map(reader)    
winRate = matches.rdd.flatMap(mapChampionIdToWinLoss)
winRate = winRate.reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1]) )
winRate =  winRate.map(calcRate)
winRate.saveAsTextFile("/content/winResult")
winRate.saveAsPickleFile("/content/winResultPickled")
stop = timeit.default_timer()
print('Time: ', stop - start)  

In [ ]:
# f = open("/content/matchesSmall.json", "r")
# lines = f.readlines()
# i = 0
# wins  = {}
# for line in lines:
#     line = json.loads(line)
#     participants = line["participants"]
#     result = []
#     for participant in participants:
#         i += 1
#         championId = participant["championId"]
#         isWin = participant["stats"]["win"]
#         if isWin:
#             t = wins.get(f"{championId}", (0, 0))
#             newT = (t[0] + 1, t[1])
#             wins[f"{championId}"] = newT
#         else:
#             t = wins.get(f"{championId}", (0, 0))
#             newT = (t[0], t[1]+1)
#             wins[f"{championId}"] = newT
# print(wins)

In [ ]:
!cat /content/winResult/* > mergedWinResul

<h5><b>**CHAMPION PICK RATE**</b></h5>

In [ ]:
def calcChampionPickRate(x):
    count = gameCount.value
    championId  = x[0]
    pickCount = x[1]
    rate = (pickCount/count) * 100
    return (championId, f"{rate}%")

In [ ]:
if os.path.isdir("/content/pickResult"):
    !rm -r /content/pickResult
if os.path.isdir("/content/pickResultPickled"):
    !rm -r /content/pickResultPickled
pickRate = matches.rdd.flatMap(lambda match: [(participant["championId"], 1) for participant in json.loads(match.value)["participants"]])
pickRate = pickRate.reduceByKey(lambda a, b: (a+b))
pickRate = pickRate.map(calcChampionPickRate)
pickRate.saveAsTextFile("/content/pickResult")
pickRate.saveAsPickleFile("/content/pickResultPickled")

In [ ]:
!cat /content/pickResult/* > mergedPickResult

<h5><b>**CHAMPION BAN RATE**</b></h5>

In [ ]:
def calcChampionBanRate(x):
    count = gameCount.value
    championId  = x[0]
    banCount = x[1]
    rate = (banCount/count) * 100
    return (championId, f"{rate}%")


In [ ]:
start = timeit.default_timer()
if os.path.isdir("/content/championBanRateResult"):
    !rm -r /content/championBanRateResult
if os.path.isdir("/content/championBanRateResultPickled"):
    !rm -r /content/championBanRateResultPickled
banRate = matches.rdd.flatMap(lambda match: [(member["championId"],1) for team in json.loads(match.value)["teams"] for member in team["bans"]])
banRate = banRate.reduceByKey(lambda a, b: (a+b))
banRate = banRate.map(calcChampionBanRate)
banRate.saveAsTextFile("/content/championBanRateResult")
banRate.saveAsPickleFile("/content/championBanRateResultPickled")
stop = timeit.default_timer()
print('Time: ', stop - start) 

In [ ]:
!cat /content/championBanRateResult/* > mergedBanResult

<h5><b>**CHAMPION DUO WIN RATE**</b></h5>

In [ ]:
def mapTwoChampionIdsToWinLoss(x):
    x = json.loads(x.value)
    participants = x["participants"]
    result = []
    for i, participant in enumerate(participants):
        for j in range(i+1, len(participant)):
            if participant["teamId"] == participants[j]["teamId"]:
                firstChampionId = participant["championId"]
                secondChampionId = participants[j]["championId"]
                isWin = participant["stats"]["win"]
                if isWin:
                    result.append(((firstChampionId, secondChampionId), (1, 0)))
                else:
                    result.append(((firstChampionId, secondChampionId), (0, 1)))
    return result

def calcRateForDuo(x):
    rate = (x[1][0]/ (x[1][0]+x[1][1])) * 100
    rate = f"{rate}%"
    return (x[0], rate)

In [ ]:
if os.path.isdir("/content/duoResult"):
    !rm -r /content/duoResult
if os.path.isdir("/content/duoResultPickled"):
    !rm -r /content/duoResultPickled
duoWinRate = matches.rdd.flatMap(mapTwoChampionIdsToWinLoss)
duoWinRate = duoWinRate.reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1]))
duoWinRate = duoWinRate.map(calcRateForDuo) 
duoWinRate.saveAsTextFile("/content/duoResult")
duoWinRate.saveAsPickleFile("/content/duoResultPickled")

In [ ]:
!cat /content/duoResult/* > mergedDuoResult

<h5><b>**ITEM WIN RATE**</b></h5>

In [ ]:
def mapItemToWinLoss(x):
    x = json.loads(x.value)
    participants = x["participants"]
    results = []
    for i, participant in enumerate(participants):
        items = []
        for j in range(7):
            # return participant["stats"][f"item{j}"]["name"]
            if participant["stats"][f"item{j}"].get("name", 0) != 0:
                items.append(participant["stats"][f"item{j}"]["name"]) 
            
        isWin = participant["stats"]["win"]
        if isWin:
            results.extend([(n, (1, 0)) for n in items])
        else:
            results.extend([(n, (0, 1)) for n in items])
    return results
def calcRateForItemWin(x):
    rate = (x[1][0]/ (x[1][0]+x[1][1])) * 100
    rate = f"{rate}%"
    return (x[0], rate)

In [ ]:
if os.path.isdir("/content/itemWinResult"):
    !rm -r /content/itemWinResult
if os.path.isdir("/content/itemWinResultPickled"):
    !rm -r /content/itemWinResultPickled
itemWinRate = matches.rdd.flatMap(mapItemToWinLoss)
itemWinRate = itemWinRate.reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1]) )
itemWinRate =  itemWinRate.map(calcRateForItemWin)
itemWinRate.saveAsTextFile("/content/itemWinResult")
itemWinRate.saveAsPickleFile("/content/itemWinResultPickled")

In [ ]:
!cat /content/itemWinResult/* > mergedItemWinResult

<h5><b>**ITEM PICK RATE**</b></h5>

In [ ]:
def itemPick(x):
    x = json.loads(x.value)
    participants = x["participants"]
    result = []
    for i, participant in enumerate(participants):
        items = []
        for j in range(7):
            if participant["stats"][f"item{j}"].get("name", 0) != 0:
                items.append(participant["stats"][f"item{j}"]["name"])
        result.extend([(item, 1) for item in items])
    return result

def itemPickRateCalc(x):
    count = gameCount.value
    item = x[0]
    itemPicks = x[1]
    rate = (itemPicks / count) * 100
    return (item, f"{rate}%")

In [ ]:
if os.path.isdir("/content/itemPickResult"):
    !rm -r /content/itemPickResult
if os.path.isdir("/content/itemPickResultPickled"):
    !rm -r /content/itemPickResultPickled
itemPickRate = matches.rdd.flatMap(itemPick)
itemPickRate = itemPickRate.reduceByKey(lambda a, b: a+b)
itemPickRate = itemPickRate.map(itemPickRateCalc)
itemPickRate.saveAsTextFile("/content/itemPickResult")
itemPickRate.saveAsPickleFile("/content/itemPickResultPickled")

In [ ]:
!cat /content/itemPickResult/* > mergedItemPickResult

<h5><b>**Item Duo with champion**</b></h5>

In [ ]:
def itemDuoWithChampion(x):
    x = json.loads(x.value)
    participants = x["participants"]
    result = [] 
    for i, participant in enumerate(participants):
        championId = participant["championId"]
        for j in range(7):
            if participant["stats"][f"item{j}"].get("name", 0) != 0:
                item = participant["stats"][f"item{j}"]["name"]
                isWin = participant["stats"]["win"]
                if isWin:
                    result.append(((championId, item), (1, 0)))
                else:
                    result.append(((championId, item), (0, 1)))
        return result

def calcRateForChampionItemDuo(x):
    rate = (x[1][0]/ (x[1][0]+x[1][1])) * 100
    rate = f"{rate}%"
    return (x[0], rate)

In [ ]:
if os.path.isdir("/content/championItemDuoWinResult"):
    !rm -r /content/championItemDuoWinResult
if os.path.isdir("/content/championItemDuoWinResultPickled"):
    !rm -r /content/championItemDuoWinResultPickled
championItemDuoWinRate = matches.rdd.flatMap(itemDuoWithChampion)
championItemDuoWinRate = championItemDuoWinRate.reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1]))
championItemDuoWinRate = championItemDuoWinRate.map(calcRateForChampionItemDuo)
championItemDuoWinRate.saveAsTextFile("/content/championItemDuoWinResult")
championItemDuoWinRate.saveAsPickleFile("/content/championItemDuoWinResultPickled")

In [ ]:
!cat /content/championItemDuoWinResult/* > mergedChampionItemDuoWinResult

<h5><b>**Item Duo with class**</b></h5>

In [ ]:
import requests

r = requests.get('http://ddragon.leagueoflegends.com/cdn/10.9.1/data/en_US/champion.json')
championsDatajson = r.json()
newChampionsDataJson = {}
for championkey, value in championsDatajson["data"].items():
    newChampionsDataJson[f"{championkey.lower().title()}"] = value
championsData = spark.sparkContext.broadcast(newChampionsDataJson)

In [ ]:
# import re
# championId = "Dr.Mundo"
# if "'" in championId:
#     newChampionId = re.split("'| ", championId)
#     newChampionId[-1] = newChampionId[-1].lower()
#     newChampionId = "".join(newChampionId)
#     print(newChampionId)
# elif " & " in championId:
#     newChampionId = re.split("'| ", championId)[0]
#     print(newChampionId)
# elif " " in championId:
#     newChampionId = re.sub("\s+", "", championId).title()
#     print(newChampionId)
# elif "." in championId:
#     newChampionId = championId.split(".")
#     newChampionId = "".join(newChampionId)
#     print(newChampionId)

In [ ]:
import re

def itemDuoWithClass(x):
    x = json.loads(x.value)
    participants = x["participants"]
    result = [] 
    for i, participant in enumerate(participants):
        championId = participant["championId"]
        
        if "'" in championId:
            newChampionId = re.split("'| ", championId)
            newChampionId[-1] = newChampionId[-1].lower()
            newChampionId = "".join(newChampionId)
        elif "." in championId:
             newChampionId = championId.replace(". ", "")
            #  newChampionId = "".join(newChampionId)
        elif " & " in championId:
            newChampionId = re.split("'| ", championId)[0]
        elif " " in championId:
            newChampionId = re.sub("\s+", "", championId)
        elif championId == "Wukong":
            newChampionId = "MonkeyKing"
        else:
            newChampionId = championId
        newChampionId = newChampionId.title()
        tags = championsData.value[f"{newChampionId}"]["tags"]
        for tag in tags:
            for i in range(7):
                if participant["stats"][f"item{i}"].get("name", 0) != 0:
                    item = participant["stats"][f"item{i}"]["name"]
                    isWin = participant["stats"]["win"]
                    if isWin:
                        result.append(((tag, item), (1, 0)))
                    else:
                        result.append(((tag, item), (0, 1)))
        return result

def calcRateForClassItemDuo(x):
    rate = (x[1][0]/ (x[1][0]+x[1][1])) * 100
    rate = f"{rate}%"
    return (x[0], rate)

In [ ]:
if os.path.isdir("/content/classItemDuoWinResult"):
    !rm -r /content/classItemDuoWinResult
if os.path.isdir("/content/classItemDuoWinResultPicked"):
    !rm -r /content/classItemDuoWinResultPicked
classItemDuoWinRate = matches.rdd.flatMap(itemDuoWithClass)
classItemDuoWinRate = classItemDuoWinRate.reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1]))
classItemDuoWinRate = classItemDuoWinRate.map(calcRateForClassItemDuo)
classItemDuoWinRate.saveAsTextFile("/content/classItemDuoWinResult")
classItemDuoWinRate.saveAsPickleFile("/content/classItemDuoWinResultPicked")

In [ ]:
!cat /content/classItemDuoWinResult/* > mergedClassItemDuoWinResult

<h5><b>**Creative Requirement**</b></h5>
<p>We will try to predict the most amusable games to watch and that will dependen on the following the higher the duo win rate between the two teams the stronger the fight will so we will calculate the syrnegy between  each pair of the five teams then we will multiply all the probabilites by each other and the most amusable match is the one that has a very high value for both teams</p>

In [ ]:
def cahmpionWinRateCreative(x):
    x = json.loads(x.value)
    participants = x["participants"]
    result = []
    for participant in participants:
        championId = participant["championId"]
        isWin = participant["stats"]["win"]
        if isWin:
            result.append((championId, (1, 0)))
        else:
            result.append((championId, (0, 1)))
    return result
def calcRateCreateive(x):
    rate = (x[1][0]/ (x[1][0]+x[1][1])) * 100
    # rate = f"{rate}%"
    return (x[0], rate)

In [ ]:
winRateCalcOp = matches.rdd.flatMap(cahmpionWinRateCreative)
winRateCalcOp = winRateCalcOp.reduceByKey(lambda a,b: (a[0]+b[0],a[1]+b[1]) )
winRateCalcOp =  winRateCalcOp.map(lambda values: (values[0], (values[1][0]/ (values[1][0]+values[1][1]))))
winRateCalcOpResult = winRateCalcOp.collect()
winRateCalcDictionary = {tuple[0]:tuple[1] for tuple in winRateCalcOpResult}

In [ ]:
print(winRateCalcDictionary)

In [ ]:
winRateSpark = spark.sparkContext.broadcast(winRateCalcDictionary)

In [ ]:
def calcWinRateForTeam(x):
    x = json.loads(x.value)
    blueTeamWinProb = 1
    redTeamWinProb = 1
    participants = x["participants"]
    gameId = x["gameId"]
    for participant in participants:
        championWinProbability = winRateSpark.value[participant["championId"]]
        championTeam = participant["teamId"]
        if championTeam == "Blue":
            blueTeamWinProb *= championWinProbability
        else:
            redTeamWinProb *= championWinProbability
    return (gameId, (blueTeamWinProb, redTeamWinProb))

In [ ]:
if os.path.isdir("/content/creative"):
    !rm -r /content/creative
if os.path.isdir("/content/creativePickled"):
    !rm -r /content/creativePickled
creative = matches.rdd.map(calcWinRateForTeam)
creative.saveAsTextFile("/content/creative")
creative.saveAsPickleFile("/content/creativePickled")

In [ ]:
!cat /content/creative/* > mergedCreative

<h5><b>**Item Suggestion**</b></h5>

<h6><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Count Approach**</b></h6>

In [ ]:
!pip install sparkpickle

In [ ]:
itemCountOp = matches.rdd.flatMap(itemPick)
itemCountOp = itemCountOp.map(lambda x: x).keys().distinct().collect()
itemCountOpLen = len(itemCountOp)
itemsCount = spark.sparkContext.broadcast(itemCountOpLen)

In [ ]:
itemToIndex = {item: i for i, item in enumerate(itemCountOp)}
itemToIndexSpark = spark.sparkContext.broadcast(itemToIndex)
indexToItem = {value: key for key, value in itemToIndex.items()}

In [ ]:
print(itemToIndexSpark.value)
print(indexToItem)

In [ ]:
import numpy as np
def championToItem(x):
    x = json.loads(x.value)
    participants = x["participants"]
    result = []
    for participant in participants:
        championId = participant["championId"]
        itemsProps = np.zeros((7, itemsCount.value))
        for j in range(7):
            if participant["stats"][f"item{j}"].get("name", 0) != 0:
                item = participant["stats"][f"item{j}"]["name"]
                isWin = participant["stats"]["win"]
                if isWin:
                    itemsProps[j][itemToIndexSpark.value[str(item)]] = 1
        result.append((championId, itemsProps))
        return result

def calcProp(x):
    for i, row in enumerate(x[1]):
        rowSum = np.sum(row)
        x[1][i] = row/rowSum
    return x

In [ ]:
if os.path.isdir("/content/suggestionProbabilityResult"):
    !rm -r /content/suggestionProbabilityResult
suggestion = matches.rdd.flatMap(championToItem)
suggestion = suggestion.reduceByKey(lambda a,b: a+b)
suggestion = suggestion.map(calcProp)
suggestion.saveAsPickleFile("/content/suggestionProbabilityResult")

In [ ]:
!cat /content/suggestionProbabilityResult/* > mergedItemSuggestionProbability

In [ ]:
if os.path.isdir("/content/suggestionProbabilityResultNotPickled"):
    !rm -r /content/suggestionProbabilityResultNotPickled
suggestionNoPickle = matches.rdd.flatMap(championToItem)
suggestionNoPickle = suggestionNoPickle.reduceByKey(lambda a,b: a+b)
suggestionNoPickle = suggestionNoPickle.map(calcProp)
suggestionNoPickle.saveAsTextFile("/content/suggestionProbabilityResultNotPickled")

In [ ]:
!cat /content/suggestionProbabilityResultNotPickled/* > mergedSuggestionProbabilityResultNotPickled

In [ ]:
# import pickle
import sparkpickle
def predictItemsForChampion(championId):
    championIdToItemProb = {}
    suggestionsNames = []
    suggestionsProbabilites = [] 
    for (dirpath, dirnames, filenames) in os.walk("/content/suggestionProbabilityResult"):
        for file in filenames:
            if not file == "_SUCCESS" and  not ".crc" in file : 
                with open(f"{dirpath}/{file}", "rb") as f:
                    for obj in sparkpickle.load(f):
                        championIdToItemProb[f"{obj[0]}"] = obj[1]
    for row in championIdToItemProb.get(championId, []):
        suggestionsNames.append([indexToItem[row.argsort()[i]] for i in range(-21, 0)])
        suggestionsProbabilites.append([row[row.argsort()[i]] for i in range(-21, 0)])
        # print(f"{indexToItem[row.argsort()[-1]]} => {row[row.argsort()[-1]]}, {indexToItem[row.argsort()[-2]]} => {row[row.argsort()[-2]]},, {indexToItem[row.argsort()[-3]]} => {row[row.argsort()[-3]]}")
    return suggestionsNames, suggestionsProbabilites
# predictItemsForChampion("Singed")
# print("\n*---------------------------------*\n")
suggestionsNames, suggestionsProbabilites = predictItemsForChampion("Lux")
# print("\n*---------------------------------*\n")
# predictItemsForChampion("Yorick")

<h6><b>**Count but conditioned on the opponents and team members classes**</b></h6>

In [ ]:
import numpy as np

def getChampionClasses(championId):
    if "'" in championId:
        newChampionId = re.split("'| ", championId)
        newChampionId[-1] = newChampionId[-1].lower()
        newChampionId = "".join(newChampionId)
    elif "." in championId:
            newChampionId = championId.replace(". ", "")
        #  newChampionId = "".join(newChampionId)
    elif " & " in championId:
        newChampionId = re.split("'| ", championId)[0]
    elif " " in championId:
        newChampionId = re.sub("\s+", "", championId)
    elif championId == "Wukong":
        newChampionId = "MonkeyKing"
    else:
        newChampionId = championId
    newChampionId = newChampionId.title()
    tags = championsData.value[f"{newChampionId}"]["tags"]
    return tags

def championToItem(x):
    x = json.loads(x.value)
    participants = x["participants"]
    result = []
    otherClasses = []
    for participant in participants:
        
        championId = participant["championId"]
        for subparticipant in participants:
            if subparticipant["championId"] != championId:
                for tag in getChampionClasses(subparticipant["championId"]):
                    if  tag not in otherClasses: 
                        otherClasses.append(tag)
        # otherClasses = set(otherClasses)
        itemsProps = np.zeros((7, itemsCount.value))
        for j in range(7):
            if participant["stats"][f"item{j}"].get("name", 0) != 0:
                item = participant["stats"][f"item{j}"]["name"]
                isWin = participant["stats"]["win"]
                if isWin:
                    itemsProps[j][itemToIndexSpark.value[str(item)]] = 1
        result.append(((championId,otherClasses), itemsProps))
        otherClasses = []
    return result

def calcProp(x):
    for i, row in enumerate(x[1]):
        rowSum = np.sum(row)
        x[1][i] = row/rowSum
    return x

In [ ]:
if os.path.isdir("/content/suggestionProbabilityWithClassesResult"):
    !rm -r /content/suggestionProbabilityWithClassesResult
suggestion = matches.rdd.flatMap(championToItem)
suggestion = suggestion.reduceByKey(lambda a,b: a+b)
suggestion = suggestion.map(calcProp)
suggestion.saveAsPickleFile("/content/suggestionProbabilityWithClassesResult")

<h6><b>**Machine Learning Approach**</b></h6>

In [ ]:
r = requests.get('http://ddragon.leagueoflegends.com/cdn/10.9.1/data/en_US/item.json')
itemsJson = r.json()

In [ ]:
itemNameToNum = {itemsJson["data"][str(item)]["name"]:item  for item in itemsJson["data"]}
itemNameToNumSpark = spark.sparkContext.broadcast(itemNameToNum)

In [ ]:
def getChampionsIds(x):
    champions = []
    x = json.loads(x.value)
    participants = x["participants"]
    for participant in participants:
        champions.append(participant["championId"])
    return champions

In [ ]:
dataFiltered = matches.rdd.flatMap(getChampionsIds)
dataFiltered = dataFiltered.distinct().collect()
# pr?int(dataFiltered)
championToId = {currentChampionId: i+1 for i, currentChampionId in enumerate(dataFiltered)}
idToChampion = {value:key for key, value in championToId.items()}
championToIdSpark = spark.sparkContext.broadcast(championToId)

In [ ]:
def dataFiltering(x):
    x = json.loads(x.value)
    participants = x["participants"]
    results = []
    for participant in participants:
        isWin = participant["stats"]["win"]
        if isWin:
            championId =  championToIdSpark.value[str(participant["championId"])]
            assists = participant["stats"]["assists"]
            level = participant["stats"]["champLevel"]
            goldEarned = participant["stats"]["goldEarned"]
            goldSpent = participant["stats"]["goldSpent"]
            damageDealtToObjectives = participant["stats"]["damageDealtToObjectives"]
            damageDealtToTurrets = participant["stats"]["damageDealtToTurrets"]
            damageSelfMitigated = participant["stats"]["damageSelfMitigated"]
            deaths = participant["stats"]["deaths"]
            doubleKills = participant["stats"]["doubleKills"]
            inhibitorKills = participant["stats"]["inhibitorKills"]
            totalDamageDealt = participant["stats"]["totalDamageDealt"]
            totalDamageDealtToChampions = participant["stats"]["totalDamageDealtToChampions"]
            totalDamageTaken = participant["stats"]["totalDamageTaken"]
            totalHeal = participant["stats"]["totalHeal"]
            totalMinionsKilled = participant["stats"]["totalMinionsKilled"]
            item0 = itemNameToNumSpark.value.get(str(participant["stats"]["item0"].get("name", "None")), -1)
            item1 = itemNameToNumSpark.value.get(str(participant["stats"]["item1"].get("name", "None")), -1)
            item2 = itemNameToNumSpark.value.get(str(participant["stats"]["item2"].get("name", "None")), -1)
            item3 = itemNameToNumSpark.value.get(str(participant["stats"]["item3"].get("name", "None")), -1)
            item4 = itemNameToNumSpark.value.get(str(participant["stats"]["item4"].get("name", "None")), -1)
            item5 = itemNameToNumSpark.value.get(str(participant["stats"]["item5"].get("name", "None")), -1)
            item6 = itemNameToNumSpark.value.get(str(participant["stats"]["item6"].get("name", "None")), -1)
            line = f"{championId},{assists},{level},{goldEarned-goldSpent},{damageDealtToObjectives},{damageDealtToTurrets},{damageSelfMitigated},{deaths},{doubleKills},{inhibitorKills},{totalDamageDealt},{totalDamageDealtToChampions},{totalDamageTaken},{totalHeal},{totalMinionsKilled},{item0},{item1},{item2},{item3},{item4},{item5},{item6}"
            results.append(line)
    return results


In [ ]:
if os.path.isdir("/content/dataFiltering"):
    !rm -r /content/dataFiltering
dataFiltered = matches.rdd.flatMap(dataFiltering)
dataFiltered.saveAsTextFile("/content/dataFiltering")

In [ ]:
topLine = "championId,assists,level,gold,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,deaths,doubleKills,inhibitorKills,totalDamageDealt,totalDamageDealtToChampions,totalDamageTaken,totalHeal,totalMinionsKilled,item0,item1,item2,item3,item4,item5,item6"
with open("/content/dataFiltering/part-00000", 'r+') as f:
    content = f.read()
    f.seek(0, 0)
    f.write(topLine + '\n' + content)

In [ ]:
import pandas as pd  

df = pd.read_csv("/content/dataFiltering/part-00000", delimiter=",")
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.1)

In [ ]:
train_features = train.drop(["item0", "item1", "item2", "item3", "item4", "item5", "item6"], axis=1)
train_features.mean(axis=0)

In [ ]:
train_features["assists"]  = ( train_features["assists"] - train_features["assists"].min() ) / (train_features["assists"].max() - train_features["assists"].min())
train_features["level"]  = ( train_features["level"] - train_features["level"].min() ) / (train_features["level"].max() - train_features["level"].min())
train_features["gold"]  = ( train_features["gold"] - train_features["gold"].min() ) / (train_features["gold"].max() - train_features["gold"].min())
train_features["damageDealtToObjectives"]  = ( train_features["damageDealtToObjectives"] - train_features["damageDealtToObjectives"].min() ) / (train_features["damageDealtToObjectives"].max() - train_features["damageDealtToObjectives"].min())
train_features["damageDealtToTurrets"]  = ( train_features["damageDealtToTurrets"] - train_features["damageDealtToTurrets"].min() ) / (train_features["damageDealtToTurrets"].max() - train_features["damageDealtToTurrets"].min())
train_features["damageSelfMitigated"]  = ( train_features["damageSelfMitigated"] - train_features["damageSelfMitigated"].min() ) / (train_features["damageSelfMitigated"].max() - train_features["damageSelfMitigated"].min())
train_features["deaths"]  = ( train_features["deaths"] - train_features["deaths"].min() ) / (train_features["deaths"].max() - train_features["deaths"].min())
train_features["doubleKills"]  = ( train_features["doubleKills"] - train_features["doubleKills"].min() ) / (train_features["doubleKills"].max() - train_features["doubleKills"].min())
train_features["inhibitorKills"]  = ( train_features["inhibitorKills"] - train_features["inhibitorKills"].min() ) / (train_features["inhibitorKills"].max() - train_features["inhibitorKills"].min())
train_features["totalDamageDealt"]  = ( train_features["totalDamageDealt"] - train_features["totalDamageDealt"].min() ) / (train_features["totalDamageDealt"].max() - train_features["totalDamageDealt"].min())
train_features["totalDamageDealtToChampions"]  = ( train_features["totalDamageDealtToChampions"] - train_features["totalDamageDealtToChampions"].min() ) / (train_features["totalDamageDealtToChampions"].max() - train_features["totalDamageDealtToChampions"].min())
train_features["totalDamageTaken"]  = ( train_features["totalDamageTaken"] - train_features["totalDamageTaken"].min() ) / (train_features["totalDamageTaken"].max() - train_features["totalDamageTaken"].min())
train_features["totalHeal"]  = ( train_features["totalHeal"] - train_features["totalHeal"].min() ) / (train_features["totalHeal"].max() - train_features["totalHeal"].min())
train_features["totalMinionsKilled"]  = ( train_features["totalMinionsKilled"] - train_features["totalMinionsKilled"].min() ) / (train_features["totalMinionsKilled"].max() - train_features["totalMinionsKilled"].min())
train_features.head()

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF



<h6><b>**Visualization**</b></h6>

In [ ]:
import matplotlib.pyplot as plt
import sparkpickle

In [ ]:
def getArrayOfTuples(pathToPicklePath):
    arrayOfTuples = []
    for (dirpath, dirnames, filenames) in os.walk(pathToPicklePath):
        for file in filenames:
            if not file == "_SUCCESS" and  not ".crc" in file : 
                with open(f"{dirpath}/{file}", "rb") as f:
                    arrayOfTuples.extend(sparkpickle.load(f))
                    # for obj in sparkpickle.load(f):
                    #     championIdToItemProb[f"{obj[0]}"] = obj[1]
    return arrayOfTuples

championWinRates = getArrayOfTuples("/content/winResultPickled")
championBanRates = getArrayOfTuples("/content/championBanRateResultPickled")
championItemDuosWinRates = getArrayOfTuples("/content/championItemDuoWinResultPickled")
classItemDuoWinRates = getArrayOfTuples("/content/classItemDuoWinResultPicked")
championDuoWinRates = getArrayOfTuples("/content/duoResultPickled")
itemPickRates = getArrayOfTuples("/content/itemPickResultPickled")
championPickRates = getArrayOfTuples("/content/pickResultPickled")
itemWinRates = getArrayOfTuples("/content/itemWinResultPickled")

In [ ]:
def visualizeLowestAndHighestTenBarChart(ArrayOfTuples, title):
    ArrayOfTuples.sort(key=lambda tup: float(tup[1].strip("%")))
    key = [str(tuple[0]) for tuple in ArrayOfTuples]
    values = [tuple[1] for tuple in ArrayOfTuples]
    plt.figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
    plt.xticks(rotation=90)
    plt.title(f"Lowest 10 {title}")
    plt.bar(key[:10], values[:10],width=0.4)
    plt.figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
    plt.xticks(rotation=90)
    plt.title(f"Highest 10 {title}")
    plt.bar(key[-10:-1], values[-10:-1],width=0.4)

In [ ]:
visualizeLowestAndHighestTenBarChart(championWinRates, "Champion Win Rates")
visualizeLowestAndHighestTenBarChart(championBanRates, "Champion Ban Rates")
visualizeLowestAndHighestTenBarChart(championItemDuosWinRates, "Champion Item Duo Win Rates")
visualizeLowestAndHighestTenBarChart(classItemDuoWinRates, "Class Item Duo Win Rates")
visualizeLowestAndHighestTenBarChart(championDuoWinRates, "Champions Duo Win Rates")
visualizeLowestAndHighestTenBarChart(itemPickRates, "Item Pick Rates")
visualizeLowestAndHighestTenBarChart(championPickRates, "Champion Pick Rates")
visualizeLowestAndHighestTenBarChart(itemWinRates, "Item Win Rates")

In [ ]:
print(len(championDuoWinRates))
print(len(championItemDuosWinRates))
print(len(classItemDuoWinRates))

In [ ]:
championDuoWinRates.sort(key=lambda tup: float(tup[1].strip("%")))
key = [str(tuple[0]) for tuple in championDuoWinRates]
values = [tuple[1] for tuple in championDuoWinRates]
plt.figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
plt.xticks(rotation=90)
plt.title(f"Champion Duo Win Rate")
plt.bar(key[10000:10020], values[10000:10020],width=0.4)

In [ ]:
championItemDuosWinRates.sort(key=lambda tup: float(tup[1].strip("%")))
key = [str(tuple[0]) for tuple in championItemDuosWinRates]
values = [tuple[1] for tuple in championItemDuosWinRates]
plt.figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
plt.xticks(rotation=90)
plt.title(f"Champion Item Duo Win Rate")
plt.bar(key[10100:10120], values[10100:10120],width=0.4)

In [ ]:
#suggestionsNames, suggestionsProbabilites
plt.figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
plt.xticks(rotation=90)
plt.title(f"item 0 suggestion for Lux")
plt.bar(suggestionsNames[0], suggestionsProbabilites[0],width=0.4)

In [ ]:
plt.figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
plt.xticks(rotation=90)
plt.title(f"item 1 suggestion for Lux")
plt.bar(suggestionsNames[1], suggestionsProbabilites[1],width=0.4)

In [ ]:
Smatches.printSchema()
# matches.filter("_corrupt_record".isNotNull).count()